In [41]:
import openai
import pandas as pd

In [42]:
client = openai.Client()

In [60]:
file = client.files.create(
    file=open("variacao_dolar_2024.csv", "rb"),
    purpose="assistants"
)

In [44]:
print(file.id)

file-AhwveFsEGpQUomeckpwWF8


In [61]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [62]:
# pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico a partir da variação do dolar ao longo de 2024"

In [63]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [64]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [65]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [66]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_FjDksGPWKpqfHZtehiw7MArL', assistant_id='asst_4WKd17Zc1NfVdxiGNau790t0', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-KE1gSkoLMRmtdLgbjp9BmP', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico que mostra a variação da cotação média do dólar ao longo de 2024. O gráfico ilustra a tendência da cotação durante os meses do ano, com uma linha conectando as médias mensais. Se precisar de mais alguma análise ou outro tipo de gráfico, fique à vontade para pedir.'), type='text')], created_at=1747678081, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_hkvapp89PVawvaok8lij4tXJ', status=None, thread_id='thread_zILAGsxKR1I6KurfVLw8KCqq'), Message(id='msg_AktRZ5id5zhDUB1ylko8nshI', assistant_id='asst_4WKd17Zc1NfVdxiGNau790t0', attachments=[], completed_at=None, content=[TextContentBl

In [ ]:
mensagens.data[0].content[0].text.value

In [67]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [68]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)


====Step message_creation
Primeiramente, vou analisar o arquivo enviado para identificar seus dados e formato. Isso nos ajudará a criar o gráfico desejado. Vamos carregar e examinar o conteúdo do arquivo.

====Step tool_calls
import pandas as pd

# Carregar o arquivo enviado
file_path = '/mnt/data/file-WWSp8QRvyCHepncJZh84Gz'

# Tentar carregar o arquivo como um DataFrame
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do DataFrame para entender sua estrutura
data.head()

====Step message_creation
O arquivo contém dados da variação do dólar ao longo de 2024, com colunas que representam o mês, o ano, a menor cotação, a maior cotação e a cotação média do dólar em reais. Vamos gerar um gráfico para visualizar a variação da cotação média do dólar ao longo do ano de 2024.

====Step tool_calls
import matplotlib.pyplot as plt

# Definir os valores do eixo x (meses) e eixo y (cotação média)
meses = data['Mês']
cotacao_media = data['Cotação Média (R$)']

# Criar o gráfico de linha
